# Using a Temporal Convolutional Network for Daytrading
## Daniel Kalam, Sharvita Paithankar

In [125]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer, Conv1D, Dropout, Dense, Activation, BatchNormalization
from tensorflow.keras.losses import CategoricalCrossentropy
import pandas as pd
from pandas_datareader import DataReader
import numpy as np
import matplotlib.pyplot as plt
import datetime

## Gathering Data
Getting data for 100 stocks in the date range of April 2nd, 2018 to October 9th, 2020 from yahoo finance.

In [126]:
symbols = ['AAPL', 'TSLA', 'FB', 'GE', 'BRK', 'GOOGL', 'INTC', 'AMD', 'HPE', 'ZM',
          'CAKE', 'AET', 'F', 'KO', 'DDS', 'NVDA', 'NFLX', 'JPM', 'AMZN', 'MSFT']
#TODO: Add 80 more symbols.
source = 'yahoo'
start_date = pd.to_datetime('2018-02-04')
end_date = pd.to_datetime('2020-10-09')
stock_data = {}
for symbol in symbols:
    stock_data[symbol] = DataReader(symbol, source, start_date, end_date)

Create a data frame for each column in a stock's data frame.

In [127]:
stock_training_data_input = np.empty((20, 676, 5))
stock_training_data_output = np.zeros((20, 676, 2))
i = 0
scaler = StandardScaler()
for symbol in stock_data:
    close_data[symbol] = stock_data[symbol].Close
    open_data[symbol] = stock_data[symbol].Open
    stock_data[symbol].drop(axis= 1, columns = ['Close'], inplace = True)
    stock_np = stock_data[symbol].to_numpy()
    stock_np = scaler.fit_transform(stock_np)
    if stock_np.shape == stock_training_data[i].shape:
        stock_training_data_input[i, :, :] = stock_np[:, :]
        for j in range(0, len(close_data[symbol])):
            if close_data[symbol][j] > open_data[symbol][j]:
                stock_training_data_output[i, j, 0] = 1
                stock_training_data_output[i, j, 1] = 0
            else:
                stock_training_data_output[i, j, 0] = 0
                stock_training_data_output[i, j, 1] = 1
        i+=1

## TensorFlow
### Converting the Data Into Tensors
Turn the data frames into tensorflow datatypes so that they can be processed by tensorflow.

In [128]:
print(stock_training_data_output[0, :10, :])
print(stock_data['AAPL'].Open[:10])
print(close_data['AAPL'][:10])
#feature_train = tf.convert_to_tensor(np.array((10, 637, 1)))
#for i in range(0, 10):
#    key = list(open_data.keys())[i]
#    feature_train[i, :, :] = tf.convert_to_tensor(np.array(stock_training_data[key]).reshape(len(open_data[key]), 5))
#open_aapl = open_data['AAPL']
#open_aapl = open_aapl[0 : 637]
#open_aapl = np.array(open_aapl).reshape(1, 637, 1)
#open_aapl_train = tf.convert_to_tensor(open_aapl)
#close_aapl = close_data['AAPL']
#close_aapl = close_aapl[0 : 637]
#close_aapl = np.array(close_aapl).reshape(1, 637, 1)

[[0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]
Date
2018-02-05    39.775002
2018-02-06    38.707500
2018-02-07    40.772499
2018-02-08    40.072498
2018-02-09    39.267502
2018-02-12    39.625000
2018-02-13    40.487499
2018-02-14    40.759998
2018-02-15    42.447498
2018-02-16    43.090000
Name: Open, dtype: float64
Date
2018-02-05    39.122501
2018-02-06    40.757500
2018-02-07    39.884998
2018-02-08    38.787498
2018-02-09    39.102501
2018-02-12    40.677502
2018-02-13    41.084999
2018-02-14    41.842499
2018-02-15    43.247501
2018-02-16    43.107498
Name: Close, dtype: float64


### Temporal Convolutional Network Class

In [129]:
filter_count = 5 # Amount of filters
final_filter_count = 2
filters = [] # Filter size for each residual block
kernel_size = 10 #Resolution of each filter
level = kernel_size
n = 0
while level <= 676:
    filters.append(filter_count)
    level+=kernel_size + (kernel_size-1)*2**n
    n+=1
filters[-1] = final_filter_count

In [130]:
class ResidualBlock(Layer):
    def __init__(self, filters, kernel_size, strides, dilation_rate, activation,
                trainable, dropout, dtype=None, activity_regularizer=None, **kwargs):
        super(ResidualBlock, self).__init__(trainable, dtype=dtype)
        self.activation = activation
        self.dropout = dropout
        self.filters = filters
        self.adjust_sample = None
        self.layer_norm = BatchNormalization(axis=-1)
        self.dilatedcausal1 = Conv1D(filters,
                                     kernel_size,
                                     strides,
                                     'causal',
                                     dilation_rate=dilation_rate)
        self.dilatedcausal2 = Conv1D(filters,
                                     kernel_size,
                                     strides,
                                     'causal',
                                     dilation_rate=dilation_rate)

    #Make the dropout based on the shape of the input
    def build(self, input_shape):
        self.drop1 = Dropout(self.dropout, input_shape)
        self.drop2 = Dropout(self.dropout, input_shape)
        if input_shape[2]!=filters:
            self.adjust_sample = Dense(self.filters)

    #The residual block processes the input
    def call(self, inputs, training):
        x = self.dilatedcausal1(inputs)
        x = self.layer_norm(x, training)
        x = self.activation(x)
        x = self.drop1(x, training) #If training is False, drop1 simply returns x
        x = self.dilatedcausal2(x)
        x = self.layer_norm(x, training)
        x = self.activation(x)
        x = self.drop2(x, training) #If training is False, drop2 simply returns x
        if self.adjust_sample is not None:
            inputs = self.adjust_sample(inputs)
        return self.activation(x+inputs)
        
class TCN(Model):
    def __init__(self, filters, kernel_size=2, dropout = 0.2, activation='relu',
                trainable=False, dtype=None, name=None,
                activity_regularizer=None, **kwargs):
        super(TCN, self).__init__()
        self.levels = []
        for i in range(0, len(filters)):
            self.levels.append(ResidualBlock(filters[i], kernel_size,
                                             1, 2**i, Activation(activation),
                                             trainable, dropout,
                                             dtype, activity_regularizer))
    
    #Running the input through each residual block
    def call(self, inputs, training=False):
        for r_block in self.levels:
            inputs = r_block(inputs, training)
        return inputs

In [134]:
tcn_model = TCN(filters, kernel_size, activation='relu', trainable = True, dtype='float')
tcn_model.compile(optimizer='adam', loss = 'mse')
tcn_model.fit(x=stock_training_data_input[0], y=stock_training_data_output[0], epochs = 25)

Epoch 1/25


StagingError: in user code:

    C:\Users\oneof\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    <ipython-input-91-bcebe246852f>:57 call  *
        inputs = r_block(inputs, training)
    C:\Users\oneof\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:982 __call__  **
        self._maybe_build(inputs)
    C:\Users\oneof\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:2643 _maybe_build
        self.build(input_shapes)  # pylint:disable=not-callable
    <ipython-input-130-bcebe246852f>:25 build
        if input_shape[2]!=filters:
    C:\Users\oneof\anaconda3\lib\site-packages\tensorflow\python\framework\tensor_shape.py:887 __getitem__
        return self._dims[key].value

    IndexError: list index out of range


In [ ]:
0/..;.x